<h4><center> AI Project Proposal Draft 2 </h4>
<h5>Ziqi Chen<br>
CS 344<br>
May 3rd, 2019</h5>



<h5>Vision — Give an overview of the project and its purpose.</h5>
This project was inspired from the neuroscience principle that our visual cortex is a complex system that designates specialized area or group of areas for the processing of different types of visual stimuli. Based on this idea, the activation patterns in our brains should be unique when we are seeing different classes of visual stimuli, such as a human face versus a cat versus an inanimate object.  
With the recent advance of neuroimaging technologies, we have more tools to shed light upon how our brain works. The brain scan data is piling up, but it isn't always easy for humans to pick apart the patterns and connections hidden in the fMRI images right away.  
In this project, I am interested to learn how to apply the strengths of Machine-Learning models we've learned about in class: pattern-extraction, to simple analysis of neuroimagine data. This project trains ML models on a classification task, one that uses a form of represented fMRI data to find hidden distinct brain activation patterns that result from seeing a class of visual stimuli, and predicts the type of stimuli the person was seeing when this brain activity was recorded. 


<h5>Background</h5>
This project aims to do a multiclass (3) classification of brain imaging samples. I'm using the [Haxby et al (2001) dataset](https://zenodo.org/record/1203329#.XNpTO0MpBqs) to perform a 3-way classification, trying to train the models to distinguish the fMRI brain scans that result from seeing a face, a cat, or a house. The dataset contains block-design fMRI data for 6 subjects who viewed 12 runs of repeated visual presentations of various stimuli. For the purpose of this project, only subjects 1 - 4 were included as they have complete fMRI data and the corresponding text labels, describing the stimuli type used in each trial. 
In terms of fMRI data manipulation, This project relies on NiLearn, a Python Scikit-Learn based library with high-level functions for manipulation and analysis of neuroimaging data. In particular, I reference the [documentation on the NiftiMasker class](http://nilearn.github.io/modules/generated/nilearn.input_data.NiftiMasker.html) which does the heavy-lifting of readying the fMRI data to be used by models. It applies a mask to 4D fMRI images to extract 2D arrays, with each datapoint representing voxel of brain region activation * time. In loadingData.py, I cited [this NiLearn example](http://nilearn.github.io/auto_examples/plot_decoding_tutorial.html#sphx-glr-auto-examples-plot-decoding-tutorial-py) to create a data matrix from the fMRIs.
In terms of technologies used, I experimented with a simple [DNN](https://github.com/kvlinden-courses/cs344-code/blob/master/u08features/keras-mnist.ipynb) and [CNN](https://github.com/kvlinden-courses/cs344-code/blob/master/u09classification/keras-cnn.ipynb) models. Neural networks, especially Convolutional Neural Networks, are known for image-processing and classification, hence the inclusion in this project.  
Last but not least, I used Linear Support Vector Machines as one of the models. Support Vector Machines are a class of supervised machine learning methods for classification and regression tasks. SVMs represent the training data as points in space (cited from [https://en.wikipedia.org/wiki/Support-vector_machine], and use a subset of training points in the decision function, also called support vectors. Its basic philosophy is to find the hyper-plane that most efficiently separates each class of samples from each other (cited from [https://www.analyticsvidhya.com/blog/2017/09/understaing-support-vector-machine-example-code/].  
From talking with Professor Vander Linden during the walkthrough, I learned that Support Vector Machines have been studied for a couple decades, and they have strong support for smaller datasets. Considering the size of this dataset, SVMs seem a good choice. Additionally, Support Vector Classification is known for its strength in classification problems that have high dimensional X inputs (cited from [https://scikit-learn.org/stable/modules/svm.html]), and that is the case in this project. It is effective not only in binary classification, but also multiclass classification problems. For the latter, SVMs supports two types of approaches: one-vs-rest or one-vs-one. One-vs-rest establishes one class and compares it against the other, whereas one-vs-one compares every class with every other, hence more computationally demanding (cited from [https://stats.stackexchange.com/questions/142325/svm-three-way-classification]. This project uses the former approach, one-vs-rest. The implementation of this model is based on [https://scikit-learn.org/stable/modules/svm.html#multi-class-classification].  
The fine-tuning of SVM hyperparameters came from documentation on the set__params method from [this page](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html). 

<h5>Implementation — Summarize your implementation and, if appropriate, how it extends on the work you’ve referenced</h5>
Given NiLearn's high-level pre-built functions, some of the implementation details will be carried out automatically in 
the library.
After reading tutorials, I think that the first step in implementation is downloading the dataset which will then be loaded
into Python. Next, I'll use NiLearn functions transforming 3D images with a time series into 2D arrays that Scikit accepts.
Then a Support Vector Machine estimator will be trained on the data for the first two tasks. The parameters will be tweaked
to achieve a satisfiable accuracy. Then, I'll need to research NiLearn functions that will plot the resulting brain activities
from the prediction result arrays. Finally, for the third task, a correlation matrix need to be created based on time series data.
A GraphicLasso estimator will be fit on the correlation matrix. I will look for existing projects on functional connectivity maps
and compare the model's prediction with their findings. 

<h5>Results — Give the results of your system and comparing them with other similar work</h5>


**Loading and Preprocessing of data**  
Here I first download the Haxby dataset and then use the NiftiMasker class from NiLearn to preprocess the 4D fMRI images into numpy matrices. 

In [ ]:
# loadingData.py employs the NiLearn Python library to download the Haxby dataset,
# which contains the brain scan images of four subjects and the accompanying labels
# of the images they looked at while their brain activity was captured.
# NiftiMasker function in the NiLearn library was used to transform the 4D brain scan images into 2D Numpy arrays,
# which are vectors in which each datapoint represents the extrapolated brain tissue voxel * time

from nilearn import datasets, image, plotting
from nilearn.input_data import NiftiMasker
from nilearn.image.image import mean_img
from nilearn.image import index_img
import numpy as np
import pandas as pd

#import Haxby et al.(2001): Faces and Objects in Ventral Temporal Cortex (fMRI)
# Subjects 5 and 6 don't have complete label or anatomical information, only included subjects 1-4
haxby_dataset = datasets.fetch_haxby(subjects=4)

#load nifti images for the given subjects. Range 0-3
#defaults to subject 2
def loadSubject(subjectNum = 1):
    # 'func' is a list of filenames: one for each subject
    fmri_filename = haxby_dataset.func[subjectNum]
    # print basic information on the dataset
    print('First subject functional nifti images (4D) are at: %s' %
          fmri_filename)  # 4D data
    return fmri_filename

#plotting subject's anatomical brain
def plotAnat(subjectNum = 1):
    path = haxby_dataset.anat[subjectNum]
    plotting.plot_stat_map(path, threshold=3)
    plotting.show()

#plotting mean functionam MRI
def plotMeanFunc(subjectNum = 1):
    mean_haxby = mean_img(haxby_dataset.func[subjectNum])
    plotting.plot_stat_map(mean_haxby, threshold=3)
    plotting.show()

#plotting one random scan of fMRI
def plotRandomFunc(subjectNum = 1):
    rand_func = index_img(haxby_dataset.func[subjectNum], 30)
    plotting.plot_stat_map(rand_func, threshold=3)
    plotting.show()

fmri_filename = loadSubject(0)
# plotAnat(subjectNum = 2)
# plotMeanFunc(2)
# plotRandomFunc(2)

behavioral = pd.read_csv(haxby_dataset.session_target[0], sep=" ")
conditions = behavioral['labels']

facecat_mask = conditions.isin(['face', 'cat'])
conditions_facecat = conditions[facecat_mask]
session_facecat = behavioral[facecat_mask].to_records(index = False)

facehouse_mask = conditions.isin(['face', 'house'])
conditions_facehouse = conditions[facehouse_mask]
session_facehouse = behavioral[facehouse_mask].to_records(index = False)

threeway_mask = conditions.isin(['face', 'house', 'cat'])
conditions_threeway = conditions[threeway_mask]
session_threeway = behavioral[threeway_mask].to_records(index = False)
# print("Number of trials: ", len(conditions_threeway))
mask_filename = haxby_dataset.mask

#masking the data from 4D image to 2D array: voxel x time
#with smothing and standardization
masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4, standardize=True, memory="nilearn_cache", memory_level=1)
X = masker.fit_transform(fmri_filename)

# Apply our condition_mask to subject 1's brain scans:
FC = X[facecat_mask]

FH = X[facehouse_mask]

FHC = X[threeway_mask]

# References
# Haxby, J., Gobbini, M., Furey, M., Ishai, A., Schouten, J., and Pietrini, P. (2001). Distributed and overlapping representations of faces and objects in ventral temporal cortex. Science 293, 2425-2430.


def processSubject(sub):
    mask_filename = haxby_dataset.mask
    # masking the data from 4D image to 2D array: voxel x time
    # with smothing and standardization
    masker = NiftiMasker(mask_img=mask_filename, smoothing_fwhm=4, standardize=True, memory="nilearn_cache",
                         memory_level=1)
    X = masker.fit_transform(loadSubject(sub))
    behavioral = pd.read_csv(haxby_dataset.session_target[sub], sep=" ")
    conditions = behavioral['labels']
    threeway_mask = conditions.isin(['face', 'house', 'cat'])
    conditions_threeway = conditions[threeway_mask]
    FHC = X[threeway_mask]
    return FHC, conditions_threeway

def processSessions(sub):
    behavioral = pd.read_csv(haxby_dataset.session_target[sub], sep=" ")
    conditions = behavioral['labels']
    threeway_mask = conditions.isin(['face', 'house', 'cat'])
    session_threeway = behavioral[threeway_mask].to_records(index = False)
    return session_threeway

X_all, Y_all = processSubject(0)
session_all =  processSessions(0)
for sub in range(1, 4):
    x, y = processSubject(sub)
    session = processSessions(sub)
    X_all = np.concatenate((X_all, x), axis = 0)
    Y_all = np.concatenate((Y_all, y))
    session_all = np.concatenate((session_all, session))
#

def peakData():
    #np.ndarray
    print("Shape of concatenated transformed fMRI data:", X_all.shape)
    print("Example row in the resulting 2D array: ", X_all[0])
    #np.series
    print("Length of concatenated labels:", Y_all.shape)
    print("First five labels: ",Y_all[0:5])
    #pd.recarray
    print("Shape of concatenated sessions: ", session_all.shape)
    print("First fifteen tuples recording sessions: ", session_all[0:15])

peakData()


**Three-way classification with simple Dense Neural Net**  
Here I use a simple neural network on the processed 2D dataset. This is a different type of ML model than the example SVM in the binary classification tutorial. 

In [ ]:
import numpy as np
from keras import models
from keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder, StandardScaler

#three-way classification with NN
X_train = X_all[:800]
X_val = X_all[800:]
# print("Length of training data: ", len(FHC_train))
# print("Lenth of validation data: ", len(FHC_val))

#need to one-hot encode the Y labels
enc = OneHotEncoder()
#cited from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
Y = enc.fit_transform(Y_all[:, np.newaxis]).toarray()
Y_train = Y[:800]
Y_val = Y[800:]

#DNN on all 4 subjects (1294 trials)
model = models.Sequential()
model.add(Dense(32, input_dim = 39912, activation='relu'))
model.add(Dense(16, input_dim = 32, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.summary()

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, Y_train, batch_size=324, epochs=10, verbose=1)
score = model.evaluate(X_val, Y_val)

print('Model with all four subjects data:')
print('Train loss:', score[0])
print('Train accuracy:', score[1])


**Three-way classification with Convolutional Neural Network**  
Again, CNN are a different class of model specializing in image classification. Here I use a form of the original brain scans instead of the masker-transformed voxel time series. 

In [1]:
from keras import layers, models
from nilearn.image import crop_img, index_img, iter_img
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def loadFilteredImages(sub):
    behavioral = pd.read_csv(haxby_dataset.session_target[sub], sep=" ")
    conditions = behavioral['labels']
    print("length of all trials: ", len(conditions))
    threeway_mask = conditions.isin(['face', 'house', 'cat'])
    images = index_img(loadSubject(sub), threeway_mask)
    return images

#for subject 1, returned images are a set of 324 frames/trials, each containing 40 slices of 64*64 images
#original shape: (40, 64, 64, 324)
subj1_images = loadFilteredImages(0)
images = np.empty((40, 64, 64))
#use np.stack to reshape the 4D image array to (324, 40, 64, 64)
images = np.stack([img.dataobj for i, img in enumerate(iter_img(subj1_images))])
train_images = images[:250]
val_images = images[250:]
print("Shape of input: ", train_images.shape)
print("Content of input: ", train_images)

#need to one-hot encode the Y labels
enc = OneHotEncoder()
#cited from https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
Y = enc.fit_transform(conditions_threeway[:, np.newaxis]).toarray()
Y_train = Y[:250]
Y_val = Y[250:]

#cited from https://github.com/fchollet/deep-learning-with-python-notebooks/blob/master/5.1-introduction-to-convnets.ipynb
model = models.Sequential()
model.add(layers.Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(40, 64, 64)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.summary()

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, Y_train, epochs=5, batch_size=20)

test_loss, test_acc = model.evaluate(val_images, Y_val)

print("Test loss: ", test_loss)
print("Test accuracy: ", test_acc)


Using TensorFlow backend.


ModuleNotFoundError: No module named 'loadingData'

**Three-way classification on Subject 1 with SVM**  
This module extends upon the tutorial I followed by doing three-way instead of binary classification. 

In [ ]:
from sklearn.feature_selection import SelectPercentile, f_classif, chi2, SelectKBest
from sklearn.svm import LinearSVC, SVC
from sklearn.pipeline import Pipeline
from nilearn import image
from nilearn.plotting import plot_stat_map, show
from sklearn.model_selection import LeaveOneGroupOut, cross_val_score
import matplotlib.pyplot as plt

#cited from: https://nilearn.github.io/auto_examples/02_decoding/plot_haxby_anova_svm.html
    # Define the dimension reduction to be used.
    # Here we use a classical univariate feature selection based on F-test,
    # namely Anova. When doing full-brain analysis, it is better to use
    # SelectPercentile, keeping 5% of voxels
    # (because it is independent of the resolution of the data).
feature_selection = SelectPercentile(f_classif, percentile=5)
k_features = SelectKBest(f_classif, k = 7)

# Output accuracy
# Define the cross-validation scheme used for validation.
# Here we use a LeaveOneGroupOut cross-validation on the session group
# which corresponds to a leave-one-session-out
def modelAccuracy(model, X, conditions, groups):
    cv = LeaveOneGroupOut()

    # Compute the prediction accuracy for the different folds (i.e. session)
    cv_scores = cross_val_score(model, X, conditions, cv=cv, groups=groups)

    # Return the corresponding mean prediction accuracy
    classification_accuracy = cv_scores.mean()

    # Print the results
    print("Classification accuracy: %.4f / Chance level: %f" %
          # (classification_accuracy, 1. / len(conditions.unique())))
          (classification_accuracy, 1. / 3))

#one-vs-the-rest linear kernel
#cited from https://scikit-learn.org/stable/modules/svm.html#multi-class-classification
#Pipeline ANOVA SVM with anova F-value, percetile feature selection. This is univariate feature selection
print("Fitting a linear SVC with Pipelined Anova f-value feature selection on subject 1 (324) trials: ")
lin_svc = LinearSVC()
facecathouse_svc = Pipeline([('anova', feature_selection), ('svc', lin_svc)])
facecathouse_svc.set_params(svc__C = 10, svc__max_iter = 2500)
facecathouse_svc.set_params(anova__percentile = 3.3, svc__max_iter = 750)
facecathouse_svc.fit(FHC, conditions_threeway)

print("Pipelined SVM with linear kernel accuracy: ")
modelAccuracy(facecathouse_svc, FHC, conditions_threeway, session_threeway)

cross_validation = cross_val_score(facecathouse_svc, FHC, conditions_threeway, cv = 6, verbose = 1)
print("Pipelined SVM with linear kernel cross validation score: ", cross_validation.mean())

**Three-way classification with SVM on all four subjects**  
This module extends upon previous work by including all four subjects with complete data in the analysis.  

In [ ]:
X_all, Y_all = processSubject(0)
session_all =  processSessions(0)
for sub in range(1, 4):
    x, y = processSubject(sub)
    session = processSessions(sub)
    X_all = np.concatenate((X_all, x), axis = 0)
    Y_all = np.concatenate((Y_all, y))
    session_all = np.concatenate((session_all, session))
# fitting on all four subjects
print("Now fitting a linear SVC on all four subjects (1296 trials) instead of one subject:")
lin_svc1 = LinearSVC()
allSubs_svc = Pipeline([('anova', feature_selection), ('svc', lin_svc1)])
allSubs_svc.set_params(anova__percentile = 2.9, svc__max_iter = 5000)
allSubs_svc.fit(X_all, Y_all)

print("Linear SVM with all subjects accuracy: ")
modelAccuracy(allSubs_svc, X_all, Y_all, session_all)

cross_validation = cross_val_score(allSubs_svc, X_all, Y_all, cv = 7, verbose = 1)
print("Linear SVM with all subjects cross validation score: ", cross_validation.mean())

<h5>Implications — Discuss the social and ethical implications of using the technologies you’ve chosen for your project</h5>
This project will be using public datasets of fMRI data.
In addition, my hope is to learn about how statistical models can offer leverage to analysis of activity and connection 
patterns in the brain. Maching learning technologies are good at pattern-finding and navigating multi-demensional datasets 
It can bring  benefits to researchers of the brain who are interested in understanding normal versus abnormal patterns of activity. 
Recently, there has been a surge of interest in using fMRI brain scans as a window to understand how our brain functions 
in a healthy or diminished way. The challenge for humans in detecting useful connections is an area of strength for 
machine learning algorithms, specifically for neural networks and statistical models that are used in the NiLearn library.
Therefore, I think these technologies will provide researchers with more insight into the underlying workings of our brain behind the mass of fMRI data. 
In the future, our understanding of neurological disorders such as Alzheimer's may stand to benefit from bringing machine learning
into the lab to assist the human medical experts. 
